# 01 报文拆分

In [1]:
import numpy as np
import pandas as pd
import binascii
import datetime
import math

In [2]:
import Ipynb_importer
from public_function import *

importing Jupyter notebook from public_function.ipynb


```
# 公共函数

# 16进制转10进制（先乘精度，后偏移）
def hex2dec(x, n=0, k=1):
    output_s = int(str(x), 16)
    output = output_s*k + n
    return output

# 16进制转字符串
def hex2str(x):
    output = binascii.a2b_hex(x).decode("utf8")
    return output

# 字符串分割
def str_split(data, n):
    global data_s, data_f, data_2
    data_s = data[:n]
    data_f = data[n:]
    data_2 = data_f[0:2]

# 时间
def bw_datetime(x):
    year = hex2dec(x[0:2]) + 2000
    month = hex2dec(x[2:4])
    day = hex2dec(x[4:6])
    hour = hex2dec(x[6:8])
    minute = hex2dec(x[8:10])
    second = hex2dec(x[10:12])
    output = datetime.datetime(year, month, day, hour, minute, second).strftime('%Y-%m-%d %H:%M:%S')
    return output

# 位数乘2
def hexlist(x):
    output = [i*2 for i in x]
    return output

# 累加，将
def hexlist2(data, n=1):
    output = [0]
    data = data * n
    for i in range(0, len(data)):
        output.append(output[i] + data[i])
    output = [i*2 for i in output]
    return output

# 
def n_m(n, m):
    n_10 = hex2dec(n)
    m_10 = hex2dec(m)
    output = n_10 * n_10
    return output

# 时间
# 采集时间 登入登出时间 共用
cf_datetime = [6]
def fun_cf_datetime(data, cf):
    cf_a = hexlist2(cf)
    data_cf_time = data[cf_a[0]:cf_a[1]]
    
    str_split(data, cf_a[-1])

    return data_cf_time
```

In [3]:
# 公共函数
# 字符串分割
data_s = ""
data_f = ""
data_m = ""
def str_split(data, n):
    global data_s, data_f, data_2
    data_s = data[:n]
    data_f = data[n:]
    data_m = data_f[0:2]

In [4]:
# 时间
# 采集时间 登入登出时间 共用
cf_datetime = [6]
def fun_cf_datetime(data, cf):
    cf_a = hexlist2(cf)
    data_cf_time = data[cf_a[0]:cf_a[1]]

    str_split(data, cf_a[-1])

    return data_cf_time

In [5]:
cf_dict = {
    "00":[2, 1, 1, 17, 1, 2],
    "00_07_01":[2, 20, 1, 1],
    "00_07_02_01":[1, 1, 1, 2, 4, 2, 2, 1, 1, 1, 2, 1, 1],
    
}

# 报文拆分

In [6]:
# 基础信息
cf_00 = [2, 1, 1, 17, 1, 2]
def fun_cf_00(data, cf):
    cf_a = hexlist2(cf)
    
    data_cf = {
        "起始符":data[cf_a[0]:cf_a[1]],
        "命令标识":data[cf_a[1]:cf_a[2]],
        "应答标志":data[cf_a[2]:cf_a[3]],
        "唯一识别码":data[cf_a[3]:cf_a[4]],
        "数据单元加密方式":data[cf_a[4]:cf_a[5]],
        "数据单元长度":data[cf_a[5]:cf_a[6]],
        "效验码":data[-2:]
    }
    str_split(data, cf_a[-1])
    return data_cf

## 数据单元

In [7]:
# 01 车辆登入
cf_01 = [2, 20, 1, 1]

def fun_cf_01(data, cf):
    cf_a = hexlist2(cf)
    
    data_cf = {
        "数据采集时间":data_cf_time,
        "登入流水号":data[cf_a[0]:cf_a[1]],
        "ICCID":data[cf_a[1]:cf_a[2]],
        "可充电储能子系统数":data[cf_a[2]:cf_a[3]],
        "可充电储能系统编码长度":data[cf_a[3]:cf_a[4]]
        # "可充电储能系统编码":data[cf_a[4]:cf_a[5]]
    }
    
    
    # 判断
    if(data_cf["可充电储能系统编码长度"]=='00'):
        data_cf["可充电储能系统编码"] = "NA"
        str_split(data, cf_a[-1])
    else:
        n = hex2dec(data_cf["可充电储能子系统数"])
        m = hex2dec(data_cf["可充电储能系统编码长度"])
        nm = n*m
        
        data_cf["可充电储能系统编码"] = data[cf_a[4]:cf_a[4] + 2*nm]
        str_split(data, cf_a[4] + 2*nm)
    
    return data_cf

In [8]:
# 02 实时数据上报

def fun_cf_02():
    global data_f, data_2, data_dict_a
    
    data_dict_a["数据单元"]["信息上报"]["数据采集时间"] = data_cf_time

    while(len(data_f)>2):
        if data_2 == '01':
            data_dict_a["数据单元"]["信息上报"]["整车数据"] = fun_cf_02_01(data_f, cf_02_01)
        elif data_2 == '02':
            data_dict_a["数据单元"]["信息上报"]["驱动电机数据"] = fun_cf_02_02(data_f, cf_02_02)
        elif data_2 == '03':
            data_dict_a["数据单元"]["信息上报"]["燃料电池数据"] = fun_cf_02_03(data_f, cf_02_03)
        elif data_2 == '04':
            data_dict_a["数据单元"]["信息上报"]["发动机数据"] = fun_cf_02_04(data_f, cf_02_04)
        elif data_2 == '05':
            data_dict_a["数据单元"]["信息上报"]["车辆位置数据"] = fun_cf_02_05(data_f, cf_02_05)
        elif data_2 == '06':
            data_dict_a["数据单元"]["信息上报"]["极值数据"] = fun_cf_02_06(data_f, cf_02_06)
        elif data_2 == '07':
            data_dict_a["数据单元"]["信息上报"]["报警数据"] = fun_cf_02_07(data_f, cf_02_07)
        elif data_2 == '08':
            data_dict_a["数据单元"]["信息上报"]["可充电储能装置电压数据"] = fun_cf_02_08(data_f, cf_02_08)
        elif data_2 == '09':
            data_dict_a["数据单元"]["信息上报"]["可充电储能装置温度数据"] = fun_cf_02_09(data_f, cf_02_09)
            break
        else :
            print("ERROR:错误的信息类型标志")
            break
    print(data_2)
    return data_dict_a

In [9]:
# 02_01 整车数据

cf_02_01 = [1, 1, 1, 2, 4, 2, 2, 1, 1, 1, 2, 1, 1]

def fun_cf_02_01(data, cf):
    cf_a = hexlist2(cf)
    # str_split(data, cf_xxsb_zhengcheshuju[-1])
    
    # 剔除 信息类型标志(1)
    data = data[2:]
    
    data_cf = {
        "车辆状态":data[cf_a[0]:cf_a[1]],
        "充电状态":data[cf_a[1]:cf_a[2]],
        "运行模式":data[cf_a[2]:cf_a[3]],
        "车速":data[cf_a[3]:cf_a[4]],
        "累计里程":data[cf_a[4]:cf_a[5]],
        "总电压":data[cf_a[5]:cf_a[6]],
        "总电流":data[cf_a[6]:cf_a[7]],
        "SOC":data[cf_a[7]:cf_a[8]],
        "DC-DC状态":data[cf_a[8]:cf_a[9]],
        "挡位":data[cf_a[9]:cf_a[10]],
        "绝缘电阻":data[cf_a[10]:cf_a[11]],
        "加速踏板行程值":data[cf_a[11]:cf_a[12]],
        "制动踏板状态":data[cf_a[12]:cf_a[13]]
    }
    str_split(data, cf_a[-1])
    return data_cf

In [10]:
# 02_02 驱动电机数据

cf_02_02 = [1, 1, 1, 2, 2, 1, 2, 2]

def fun_cf_02_02(data, cf):
    # 剔除 信息类型标志(1)
    data = data[2:]

    # 驱动电机个数
    n_dianji_o = data[0:2]
    n_dianji = hex2dec(data[0:2])

    # 剔除 电机个数(1)
    data = data[2:]

    cf_a = hexlist2(cf)

    # 个体长
    data_all = {}
    for i in range(0,n_dianji):
        data_s = {
            data[cf_a[0]:cf_a[1]]:{
                    "驱动电机状态":data[cf_a[1]:cf_a[2]],
                    "驱动电机控制器温度":data[cf_a[2]:cf_a[3]],
                    "驱动电机转速":data[cf_a[3]:cf_a[4]],
                    "驱动电机转矩":data[cf_a[4]:cf_a[5]],
                    "驱动电机温度":data[cf_a[5]:cf_a[6]],
                    "电机控制器输入电压":data[cf_a[6]:cf_a[7]],
                    "电机控制器直流母线电流":data[cf_a[7]:cf_a[8]]
            }
        }
        data_all = dict(data_all, **data_s)
        # 截取
        str_split(data, cf_a[-1])
        data = data[cf_a[-1]:]
        
    data_cf = {
        "驱动电机个数":n_dianji_o,
        "驱动电机总成信息列表":data_all
    }
    return data_cf

In [11]:
# 02_03 燃料电池数据(未完成)

cf_02_03 = [1, 4, 4]

def fun_cf_02_03(data, cf):
    # 剔除 信息类型标志(1)
    data = data[2:]
    
    cf_a = hexlist2(cf)
    
    data_cf = {        
        "燃料电池电压":data[cf_a[0]:cf_a[1]],
        "燃料电池电流":data[cf_a[1]:cf_a[2]],
        "燃料消耗率":data[cf_a[2]:cf_a[3]],
        "燃料电池温度探针总数":data[cf_a[3]:cf_a[4]],
        "探针温度值":data[cf_a[4]:cf_a[5]],
        "氢系统中最高温度":data[cf_a[5]:cf_a[6]],
        "氢系统中最高温度探针代号":data[cf_a[6]:cf_a[7]],
        "氢气最高浓度":data[cf_a[7]:cf_a[8]],
        "氢气最高浓度传感器代号":data[cf_a[8]:cf_a[9]],
        "氢气最高压力":data[cf_a[9]:cf_a[10]],
        "氢气最高压力传感器代号":data[cf_a[10]:cf_a[11]],
        "高压DC/DC状态":data[cf_a[11]:cf_a[12]],
    }
    str_split(data, cf_a[-1])
    
    return data_cf

In [12]:
# 02_04 发动机数据

cf_02_05 = [1, 2, 2]

def fun_cf_02_04(data, cf):
    # 剔除 信息类型标志(1)
    data = data[2:]
    
    cf_a = hexlist2(cf)
    
    data_cf = {
        "发动机状态":data[cf_a[0]:cf_a[1]],
        "曲轴转速":data[cf_a[1]:cf_a[2]],
        "燃料消耗率":data[cf_a[2]:cf_a[3]],
    }
    str_split(data, cf_a[-1])
    return data_cf

In [13]:
# 02_05 车辆位置数据

cf_02_05 = [1, 4, 4]

def fun_cf_02_05(data, cf):
    # 剔除 信息类型标志(1)
    data = data[2:]
    
    cf_a = hexlist2(cf)
    
    data_cf = {
        "定位状态":data[cf_a[0]:cf_a[1]],
        "经度":data[cf_a[1]:cf_a[2]],
        "纬度":data[cf_a[2]:cf_a[3]],
    }
    str_split(data, cf_a[-1])
    return data_cf

In [14]:
# 02_06 极值数据
cf_02_06 = [1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1]

def fun_cf_02_06(data, cf):
    # 剔除 信息类型标志(1)
    data = data[2:]
    
    cf_a = hexlist2(cf)
    
    data_cf = {
        "最高电压电池子系统号":data[cf_a[0]:cf_a[1]],
        "最高电压电池单体代号":data[cf_a[1]:cf_a[2]],
        "电池单体电压最高值":data[cf_a[2]:cf_a[3]],
        "最低电压电池子系统号":data[cf_a[3]:cf_a[4]],
        "最低电压电池单体代号":data[cf_a[4]:cf_a[5]],
        "电池单体电压最低值":data[cf_a[5]:cf_a[6]],
        "最高温度子系统号":data[cf_a[6]:cf_a[7]],
        "最高温度探针序号":data[cf_a[7]:cf_a[8]],
        "最高温度值":data[cf_a[8]:cf_a[9]],
        "最低温度子系统号":data[cf_a[9]:cf_a[10]],
        "最低温度探针序号":data[cf_a[10]:cf_a[11]],
        "最低温度值":data[cf_a[11]:cf_a[12]],
    }
    str_split(data, cf_a[-1])
    return data_cf

In [15]:
# 02_07 报警数据
# 此栏子故障数未使用

cf_02_07 = [1, 4, 1, 1, 1, 1]

def fun_cf_02_07(data, cf):
    # 剔除 信息类型标志(1)
    data = data[2:]
    
    cf_a = hexlist2(cf)

    data_cf = {
        "最高报警等级":data[cf_a[0]:cf_a[1]],
        "通用报警标志":data[cf_a[1]:cf_a[2]],
        "可充电储能装置故障总数N1":data[cf_a[2]:cf_a[3]],
        "驱动电机故障总数N2":data[cf_a[3]:cf_a[4]],
        "发动机故障总数N3":data[cf_a[4]:cf_a[5]],
        "其他故障总数N4":data[cf_a[5]:cf_a[6]],
    }
    str_split(data, cf_a[-1])
    return data_cf

In [16]:
# 02_08 可充电储能装置电压数据
cf_02_08 = [1, 2, 2, 2, 2, 1]

def fun_cf_02_08(data, cf):
    # 剔除 信息类型标志(1)
    data = data[2:]

    # 充电储能装置个数
    n_dianchi_o = data[0:2]
    n_dianchi = hex2dec(data[0:2])

    # 剔除 充电储能装置个数(1)
    data = data[2:]
    
    data_all = {}
    for i in range(0,n_dianchi):
        
        cf_a = hexlist2(cf)
        n_dianchi_s = hex2dec(data[cf_a[5]:cf_a[6]])
        cf_b = cf.copy()
        cf_b.append(n_dianchi_s*2)
        cf_a1 = hexlist2(cf_b)
        data_s = {
            data[cf_a1[0]:cf_a1[1]]:{
                    "可充电储能装置电压":data[cf_a1[1]:cf_a1[2]],
                    "可充电储能装置电流":data[cf_a1[2]:cf_a1[3]],
                    "单体电池总数":data[cf_a1[3]:cf_a1[4]],
                    "本帧起始电池序号":data[cf_a1[4]:cf_a1[5]],
                    "本帧单体电池总数":data[cf_a1[5]:cf_a1[6]],
                    "单体电池电压":data[cf_a1[6]:cf_a1[7]],
            }
        }
        data_all = dict(data_all, **data_s)
        # 截取
        str_split(data, cf_a1[-1])
        data = data[cf_a[-1]:]
        
    data_cf = {
        "可充电储能子系统个数":n_dianchi_o,
        "可充电储能子系统电压信息列表":data_all
    }
    return data_cf

In [17]:
# 02_09 可充电储能装置温度数据
cf_02_09 = [1, 2]

def fun_cf_02_09(data, cf):
    # 剔除 信息类型标志(1)
    data = data[2:]

    # 充电储能装置个数
    n_dianchi_o = data[0:2]
    n_dianchi = hex2dec(data[0:2])

    # 剔除 充电储能装置个数(1)
    data = data[2:]
    
    data_all = {}
    for i in range(0,n_dianchi):
        
        cf_a = hexlist2(cf)
        n_dianchi_s = hex2dec(data[cf_a[1]:cf_a[2]])
        cf_b = cf.copy()
        cf_b.append(n_dianchi_s)
        cf_a1 = hexlist2(cf_b)
        data_s = {
            data[cf_a1[0]:cf_a1[1]]:{
                    "可充电储能温度探针个数":data[cf_a1[1]:cf_a1[2]],
                    "可充电储能子系统各温度探针检测到的温度值":data[cf_a1[2]:cf_a1[3]],
            }
        }
        data_all = dict(data_all, **data_s)
        # 截取
        str_split(data, cf_a1[-1])
        data = data[cf_a1[-1]:]
        
    data_cf = {
        "可充电储能子系统个数":n_dianchi_o,
        "可充电储能子系统温度信息列表":data_all
    }
    return data_cf

In [18]:
# 03 补发报文
# 同 02 实时报文

In [19]:
# 04 车辆登出
cf_04 = [2]

def fun_cf_04(data, cf):
    cf_a = hexlist2(cf)
    
    data_cf = {
        "数据采集时间":data_cf_time,
        "登出流水号":data[cf_a[0]:cf_a[1]],
    }
    str_split(data, cf_a[-1])
    return data_cf

In [20]:
# 05 平台登入
cf_05 = [2, 12, 20, 1]

def fun_cf_05(data, cf):
    cf_a = hexlist2(cf)
    
    data_cf = {
        "平台登入时间":data_cf_time,
        "登入流水号":data[cf_a[0]:cf_a[1]],
        "平台用户名":data[cf_a[1]:cf_a[2]],
        "平台密码":data[cf_a[2]:cf_a[3]],
        "加密规则":data[cf_a[3]:cf_a[4]],
    }
    str_split(data, cf_a[-1])
    return data_cf

In [21]:
# 06 平台登出
cf_06 = [2]

def fun_cf_06(data, cf):
    cf_a = hexlist2(cf)
    
    data_cf = {
        "登出时间":data_cf_time,
        "登出流水号":data[cf_a[0]:cf_a[1]],
    }
    str_split(data, cf_a[-1])
    return data_cf

# 执行函数

In [22]:
def fun_cf(data):
    global data_dict_a, data_cf_time
    # 基础数据
    data_dict_a = fun_cf_00(data, cf_00)

    # 时间
    data_cf_time = fun_cf_datetime(data_f, cf_datetime)

    data_dict_a['数据单元'] = {}
    print("命令标识：" + data_dict_a["命令标识"])

    # 从命令标识，创建
    if data_dict_a["命令标识"] == "01":
        data_dict_a["数据单元"]["车辆登入"] = {}
        data_dict_a["数据单元"]["车辆登入"] = fun_cf_01(data_f, cf_01)
    elif data_dict_a["命令标识"] == "02" or data_dict_a["命令标识"] == "03":
        data_dict_a['数据单元']['信息上报'] = {}
        data_dict_a = fun_cf_02()
    elif data_dict_a["命令标识"] == "03":
        data_dict_a["数据单元"]["车辆登出"] = fun_cf_03(data_f, cf_03)
    elif data_dict_a["命令标识"] == "04":
        data_dict_a["数据单元"]["平台登入"] = fun_cf_04(data_f, cf_04)
    elif data_dict_a["命令标识"] == "05":
        data_dict_a["数据单元"]["平台登入"] = fun_cf_05(data_f, cf_05)
    else:
        print("ERROR:命令标识")

    if data_f == data_2:
        print("报文解析完毕，正常")
    else :
        print("报文解析完毕，异常")
        print("剩余报文：" + data_f)
        
    return data_dict_a